# Overview of std and mean maps of the template

- std
- mean
of each template network

In [6]:
import re
import os
import copy
import gzip
import numpy as np
import pandas as pd
import cPickle as cp
import brainbox as bb
import random  as rnd
import nibabel as nib
import multiprocessing as mp
from scipy import stats as st
from matplotlib import pyplot as plt
import scipy.cluster.hierarchy as clh
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [7]:
# Duplicate dropper
def drop_duplicates(in_dict):
    """
    Because python uses pointers and does not copy the variables
    I can operate directly on the dictionary and change it in place
    """
    cp_dict = copy.deepcopy(in_dict)
    subs = cp_dict['sub_name']
    dirs = cp_dict['dir']
    path = cp_dict['path']
    drop = list()
    present = list()
    sub_names = np.array([int64(re.search(r'(?<=\d{2})\d{5}', sub_id).group()) for sub_id in cp_dict['sub_name']])
    for idx, sub in enumerate(sub_names):
        if not sub in present:
            present.append(sub)
        else:
            drop.append(idx)
    print('Found {} items to drop'.format(len(drop)))
    # Pop them in reverse order
    for idx in drop[::-1]:
        subs.pop(idx)
        dirs.pop(idx)
        path.pop(idx)
    
    return cp_dict

In [8]:
# Paths
in_path = '/data1/abide/Out/Remote/all_worked/out/sc36/'
in_part = '/data1/cambridge/cambridge_new/cambridge/out/stability_group_nii/sci10_scg12_scf12/brain_partition_consensus_group_sci10_scg12_scf12.nii.gz'
out_path = '/data1/abide/Out/Remote/all_worked/out/maps/'
scale = 36

metric = 'stability_maps'
file_dict = bb.fileOps.grab_files(in_path, '.nii.gz', metric)
# Get subject IDs of the files I just read in
file_dict = drop_duplicates(file_dict)
data_subs = np.array([int64(re.search(r'(?<=\d{2})\d{5}', sub_id).group()) for sub_id in file_dict['sub_name']])

StopIteration: 

In [ ]:
# get a reference image
ref_path = file_dict['path'][0]
img = nib.load(ref_path)
data = img.get_data()
ref_shape = data.shape
n_subs = len(data_subs)
# get the reference partition
pimg = nib.load(in_part)
pdata = pimg.get_data()

# Now go through the networks and make those maps

In [1]:
network = 3

for network in np.arange(12):
    print('running network {} now'.format(network))
    net_data = np.zeros((ref_shape[:3] + (n_subs,)))
    for idx, path in enumerate(file_dict['path']):
        img = nib.load(path)
        data = img.get_data()
        net = data[..., network]
        net_data[..., idx] = net
    mean_net = np.mean(net_data, axis=3)
    std_net = np.std(net_data, axis=3)
    if network == 0:
        all_mean = mean_net[..., None]
        all_std = std_net[..., None]
    else:
        all_mean = np.concatenate((all_mean, mean_net[..., None]), axis=3)
        all_std = np.concatenate((all_std, std_net[..., None]), axis=3)
    #out_mean = nib.Nifti1Image(mean_net, pimg.get_affine(), pimg.get_header())
    #nib.save(out_mean, os.path.join(out_path, 'mean_{}.nii.gz'.format(network)))
    #out_std = nib.Nifti1Image(std_net, pimg.get_affine(), pimg.get_header())
    #nib.save(out_std, os.path.join(out_path, 'std_{}.nii.gz'.format(network)))
    
out_all_mean = nib.Nifti1Image(all_mean, pimg.get_affine(), pimg.get_header())
nib.save(out_all_mean, os.path.join(out_path, 'all_mean.nii.gz'))
out_all_std = nib.Nifti1Image(all_std, pimg.get_affine(), pimg.get_header())
nib.save(out_all_std, os.path.join(out_path, 'all_std.nii.gz'))

running network 0 now


NameError: name 'ref_shape' is not defined

In [111]:
network = 3

print('running network {} now'.format(network))
net_data = np.zeros((ref_shape[:3] + (n_subs,)))
for idx, path in enumerate(file_dict['path']):
    img = nib.load(path)
    data = img.get_data()
    net = data[..., network]
    net_data[..., idx] = net
mean_net = np.mean(net_data, axis=3)
std_net = np.std(net_data, axis=3)
out_mean = nib.Nifti1Image(mean_net, pimg.get_affine(), pimg.get_header())
nib.save(out_mean, os.path.join(out_path, 'mean_{}.nii.gz'.format(network)))
out_std = nib.Nifti1Image(std_net, pimg.get_affine(), pimg.get_header())
nib.save(out_std, os.path.join(out_path, 'std_{}.nii.gz'.format(network)))

running network 3 now


In [ ]:
# Make the overview

In [47]:
mean_net = np.mean(net_data, axis=3)
std_net = np.std(net_data, axis=3)



# Visualize 

In [103]:
x = 30
y = 30
z = 30

target = std_net

# Get the network in the template
tmp = np.zeros_like(pdata)
tmp[pdata==network] = 1
tmp_x = np.rot90(tmp[x, :, :])
tmp_y = np.rot90(tmp[:, y, :])
tmp_z = np.rot90(tmp[:, :, z])

slc_x = np.rot90(target[x, :, :])
slc_y = np.rot90(target[:, y, :])
slc_z = np.rot90(target[:, :, z])

In [ ]:
x = 30
y = 30
z = 30

target = std_net

# Get the network in the template
tmp = np.zeros_like(pdata)
tmp[pdata==network] = 1
tmp_x = np.rot90(tmp[x, :, :])
tmp_y = np.rot90(tmp[:, y, :])
tmp_z = np.rot90(tmp[:, :, z])

slc_x = np.rot90(target[x, :, :])
slc_y = np.rot90(target[:, y, :])
slc_z = np.rot90(target[:, :, z])

f = plt.figure(figsize=(20,10))
ax = f.add_subplot(231)
ay = f.add_subplot(232)
az = f.add_subplot(233)
ax.matshow(slc_x)
ay.matshow(slc_y)
az.matshow(slc_z)

tx = f.add_subplot(234)
ty = f.add_subplot(235)
tz = f.add_subplot(236)
#tx.matshow(slc_x, alpha=0.8)
#ty.matshow(slc_y, alpha=0.8)
#tz.matshow(slc_z, alpha=0.8)
tx.matshow(tmp_x, alpha=0.5)
ty.matshow(tmp_y, alpha=0.5)
tz.matshow(tmp_z, alpha=0.5)

f.tight_layout()

In [94]:
tmp.shape

(7759,)